In [2]:
import pandas as pd
import sqlite3
import csv

pd.set_option("max_columns",180)
pd.set_option("max_rows",200000)
pd.set_option("max_colwidth", 5000)

game_log = pd.read_csv("game_log.csv", low_memory = False)
park_codes = pd.read_csv("park_codes.csv")
person_codes = pd.read_csv("person_codes.csv")
team_codes = pd.read_csv("team_codes.csv")

### Description of Defensive Position numbers
- 106-132   Visiting starting players ID, name and defensive position, listed in the order (1-9) they appeared in the batting order.
- 133-159   Home starting players ID, name and defensive position listed in the order (1-9) they appeared in the batting order.

In [3]:
game_log.shape #(171907, 161)
game_log.head()
#game_log.tail()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [4]:
game_log.iloc[:,106:132].head()

,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos
0,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0
1,George Wright,6.0,barnr102,Ross Barnes,4.0,birdd102,Dave Birdsall,9.0,mcvec101,Cal McVey,2.0,wrigh101,Harry Wright,8.0,goulc101,Charlie Gould,3.0,schah101,Harry Schafer,5.0,conef101,Fred Cone,7.0,spala101,Al Spalding,1.0
2,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0
3,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0
4,George Wright,6.0,barnr102,Ross Barnes,4.0,birdd102,Dave Birdsall,9.0,mcvec101,Cal McVey,2.0,wrigh101,Harry Wright,8.0,goulc101,Charlie Gould,3.0,schah101,Harry Schafer,5.0,conef101,Fred Cone,7.0,spala101,Al Spalding,1.0


In [5]:
game_log.iloc[:,133:159].head()

,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos
0,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0
1,Fred Waterman,5.0,forcd101,Davy Force,6.0,mille105,Everett Mills,3.0,allid101,Doug Allison,2.0,hallg101,George Hall,7.0,leona101,Andy Leonard,4.0,braia102,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0
2,Denny Mack,3.0,addyb101,Bob Addy,4.0,fishc102,Cherokee Fisher,1.0,hasts101,Scott Hastings,8.0,ham-r101,Ralph Ham,5.0,ansoc101,Cap Anson,2.0,sagep101,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0
3,Bub McAtee,3.0,kingm101,Marshall King,8.0,hodec101,Charlie Hodes,2.0,woodj106,Jimmy Wood,4.0,simmj101,Joe Simmons,9.0,folet101,Tom Foley,7.0,duffe101,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0
4,Clipper Flynn,9.0,mcgem101,Mike McGeary,2.0,yorkt101,Tom York,8.0,mcmuj101,John McMullin,1.0,kings101,Steve King,7.0,beave101,Edward Beavens,4.0,bells101,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0


In [6]:
park_codes.shape #(252, 9)
park_codes.head()
park_codes.tail()

,park_id,name,aka,city,state,start,end,league,notes
247,WIL02,BB&T Ballpark at Bowman Field,NaN,Wiliamsport,PA,08/20/2017,08/20/2017,NL,PIT
248,WNY01,West New York Field Club Grounds,NaN,West New York,NJ,09/11/1898,09/17/1899,NL,"BRO:9/18&10/2/1898; NY1:9/11/98, 6/4&7/16&8/13&9/17/99"
249,WOR01,Agricultural County Fair Grounds I,NaN,Worcester,MA,05/01/1880,09/29/1882,NL,NaN
250,WOR02,Agricultural County Fair Grounds II,NaN,Worcester,MA,08/17/1887,08/17/1887,NL,1 BSN game
251,WOR03,Worcester Driving Park Grounds,NaN,Worcester,MA,10/30/1874,10/30/1874,NaN,1 BS1 game


In [7]:
park_codes.league.value_counts()

#NL    National League
#AL    American League
#AA    American Association
#UA    Union Association
#FL    Florida State League
#PL    Players League

NL    88
AL    47
AA    33
UA     9
FL     7
PL     2
Name: league, dtype: int64

In [8]:
person_codes.shape #(20494, 7)
person_codes.head()
person_codes.tail()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
20489,zuvep001,Zuvella,Paul,09/04/1982,NaN,04/02/1996,NaN
20490,zuveg101,Zuverink,George,04/21/1951,NaN,NaN,NaN
20491,zwild101,Zwilling,Dutch,08/14/1910,NaN,04/15/1941,NaN
20492,zycht001,Zych,Tony,09/04/2015,NaN,NaN,NaN
20493,thoma102,Thompson,NaN,NaN,NaN,NaN,NaN


In [9]:
team_codes.shape #(150, 8)
team_codes.head()
team_codes.tail()

,team_id,league,start,end,city,nickname,franch_id,seq
145,WS8,NL,1886,1889,Washington,Senators,WS8,1
146,WS9,AA,1891,1891,Washington,Senators,WS9,1
147,WSN,NL,1892,1899,Washington,Senators,WS9,2
148,WSU,UA,1884,1884,Washington,Nationals,WSU,1
149,MIA,NL,2012,0,Miami,Marlins,FLO,2


In [10]:
team_codes.league.value_counts() # same values as in park_codes (diff. counts)

NL    45
AL    25
AA    24
UA    13
FL     9
PL     8
Name: league, dtype: int64

In [11]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


### Recreate the `run_command()` and `run_query()` functions from the previous guided project

In [81]:
db = 'mlb.db'

def create_connection(db):
    """ create a database connection to a SQLite database """
    conn = None
    with sqlite3.connect(db) as conn:
        return conn      

def run_command(d):
    with sqlite3.connect(db) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(d)
    
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

con = create_connection(db)
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table
4,person,table
5,park,table
6,appearance_type,table
7,league,table
8,team,table
9,game,table


In [25]:
#game_log.to_sql('game_log', con, index=False)
#park_codes.to_sql('park_codes', con, index=False)
#person_codes.to_sql('person_codes', con, index=False)
#team_codes.to_sql('team_codes', con, index=False)

In [26]:
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table
4,park,table
5,league,table
6,team,table
7,game,table
8,team_appearance,table
9,appearance_type,table


In [27]:
q = '''
    
select h_name, 
       date,
       number_of_game
from game_log
limit 5
'''
    
run_query(q)

,h_name,date,number_of_game
0,FW1,18710504,0
1,WS3,18710505,0
2,RC1,18710506,0
3,CH1,18710508,0
4,TRO,18710509,0


### Creating the primary key for th game_log table
Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, `ATL198304080` should be read as follows. 
- The first three characters identify the home team (the Braves).
- The next four are the year (1983). 
- The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). 
- The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header 

The id record starts the description of a game thus ending the description of the preceding game in the file.

# creating new column without values first:


In [28]:
q = '''
select game_id from game_log
limit 5

'''

#column was created successfully:

'''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''
run_query(q)  

#fill new column with concatenated strings from h_name, date and number_of_game\n",

,game_id
0,FW1187105040
1,WS3187105050
2,RC1187105060
3,CH1187105080
4,TRO187105090


In [29]:
# code for concatenating the game_id column
'''

UPDATE game_log
SET game_id = (h_name || date || number_of_game)

'''


'\n\nUPDATE game_log\nSET game_id = (h_name || date || number_of_game)\n\n'

## Normalization
### Repetition in columns
We have three columns that relate to one player, followed by three columns that relate to another player. We could restructure our data to remove this repetition - we would need to add an extra column to include the data that was previously only contained in the name of the column:

|id|name|def_pos|off_pos|
|----|----|----|----|
|villj001|	Jonathan Villar	|5.0|	1.0
|granc001|	Curtis Granderson|	8.0|	1.0
|kendh001|	Howie Kendrick|	7.0	|1.0
|jasoj001|	John Jaso|	3.0	|1.0
|gordd002|	Dee Gordon|	4.0	|1.0
|genns001|	Scooter Gennett|	4.0|	2.0


### Non-primary key columns should be attributes of the primary key

The primary key of our game log is our `game_id`, but the players name are not attributes of a game, but of the player id. If the only data we had was the game log, we would remove this column and create a new table that had the names of each player. As it happens, our `person_codes` table already has a list of our player IDs and names, so we can remove these without the need for creating a new table first.

### Redundant Data

Lastly, we want to eliminate any redundant data - that is, columns where the data is available elsewhere. A good example of this can be found in our park_codes table, which will form the basis of our eventual park table. Let's look at the first few rows (we won't display the notes column as is not relevant to our discussion):

|park_id|	name|	aka	|city|	state|	start|	end	|league|
|----|----|----|----|----|----|----|----|
|ALB01|	Riverside Park|	NaN|	Albany|	NY|	09/11/1880|	05/30/1882|	NL|
|ALT01|	Columbia Park|	NaN	|Altoona|	PA	|04/30/1884|	05/31/1884|	UA|
|ANA01|	Angel Stadium of Anaheim|	Edison Field; Anaheim Stadium|	Anaheim|CA|	04/19/1966|	NaN	|AL|

The start and end columns show the first and last games played at the park, however we will be able to derive this information by looking at the park information for each game. Similarly, the league information is going to be available elsewhere in our database.

In [30]:
q = '''

select * from team_codes
limit 5
'''
run_query(q)

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,None,1872,1874,Baltimore,Canaries,BL1,1


In [31]:
#pitch_cols = [col for col in game_log.columns if "pitch" in col]
game_log.filter(regex = "pitch").head()

,v_hit_by_pitch,v_pitchers_used,v_wild_pitches,h_hit_by_pitch,h_pitchers_used,h_wild_pitches,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name
0,0.0,1.0,0.0,0.0,1.0,0.0,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews
1,0.0,1.0,1.0,0.0,1.0,0.0,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard
2,0.0,1.0,2.0,0.0,1.0,1.0,prata101,Al Pratt,fishc102,Cherokee Fisher,NaN,NaN,prata101,Al Pratt,fishc102,Cherokee Fisher
3,0.0,1.0,0.0,0.0,1.0,0.0,zettg101,George Zettlein,prata101,Al Pratt,NaN,NaN,prata101,Al Pratt,zettg101,George Zettlein
4,0.0,1.0,0.0,0.0,1.0,1.0,spala101,Al Spalding,mcmuj101,John McMullin,NaN,NaN,spala101,Al Spalding,mcmuj101,John McMullin


### my personal normalized db version
--> import images with `<img src="Bildschirmfoto 2020-01-11 um 11.41.50.png" title="Edited DB Schema" />`

<img src="Bildschirmfoto 2020-01-11 um 11.41.50.png" title="Edited DB Schema" />

### DQ´s version of normalized DB
<img src="mlb_schema.svg" title="DQ DB Schema" />

1. Create the person table with columns and primary key as shown in the schema diagram.
 - Select the appropriate type based on the data.
 - Insert the data from the person_codes table.
 - Write a query to display the first few rows of the table.

In [34]:
c1 = '''

DROP TABLE IF EXISTS person

'''

c = '''

CREATE TABLE IF NOT EXISTS person(
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
    
)

'''
run_command(c1)
run_command(c)

In [35]:
c = '''

INSERT OR IGNORE INTO person
SELECT id, first, last FROM person_codes
'''

run_command(c)

In [36]:
q = '''
SELECT * FROM person
LIMIT 5
'''
run_query(q)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


`2. Create the park table with columns and primary key as shown in the schema diagram.
  - Select the appropriate type based on the data
  - Insert the data from the park_codes table.
  - Write a query to display the first few rows of the table.

In [37]:
c1 = '''
DROP TABLE IF EXISTS park

'''


c = '''

CREATE TABLE IF NOT EXISTS park(
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
    
)

'''
run_command(c1)
run_command(c)

In [38]:
c = '''

INSERT OR IGNORE INTO park
SELECT park_id, name, aka, city, state, notes
FROM park_codes
'''

run_command(c)

In [39]:
q = '''
SELECT * FROM park
LIMIT 5
'''
run_query(q)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


`3. Create the league table with columns and primary key as shown in the schema diagram.
 - Select the appropriate type based on the data.
 - Insert the data manually based on your research on the names of the six league IDs.
 - Write a query to display the table.

In [47]:
c1 = '''
DROP TABLE IF EXISTS league

'''


c = '''

CREATE TABLE IF NOT EXISTS league(
    league_id TEXT PRIMARY KEY,
    name TEXT NOT NULL
    
)

'''
run_command(c1)
run_command(c)

In [48]:
#NL    National League
#AL    American League
#AA    American Association
#UA    Union Association
#FL    Florida State League
#PL    Players League

c = '''

INSERT OR IGNORE INTO league (league_id, name)
VALUES("NL", "National League"),
("AL", "American League"),
("AA", "American Association"),
("UA", "Union Association"),
("FL", "Florida State Leaague"),
("PL", "Players League");

'''

run_command(c)

In [49]:
q = '''
SELECT * FROM league

'''
run_query(q)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,UA,Union Association
4,FL,Florida State Leaague
5,PL,Players League


`4. Create the appearance_type table with columns and primary key as shown in the schema diagram.
 - Select the appropriate type based on the data.
 - Import and insert the data from appearance_type.csv.
 - Write a query to display the table.

In [43]:
appearance = pd.read_csv("appearance_type.csv")
appearance.head()

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [44]:
c1 = "DROP TABLE IF EXISTS appearance_type;"

run_command(c1)

c2 = """
CREATE TABLE appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
"""
run_command(c2)

appearance_type = pd.read_csv('appearance_type.csv')

with sqlite3.connect('mlb.db') as conn:
    appearance_type.to_sql('appearance_type',
                           conn,
                           index=False,
                           if_exists='append')

q = """
SELECT * FROM appearance_type
LIMIT 5
"""

run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


<img src="mlb_schema_2.svg" title="creating foreign keys" />

1. Create the team table with columns, primary key, and foreign key as shown in the schema diagram.
 - Select the appropriate type based on the data.
 - Insert the data from the team_codes table.
 - Write a query to display the first few rows of the table.

In [59]:
c1 = '''

DROP TABLE IF EXISTS team
'''

c= '''
CREATE TABLE IF NOT EXISTS team(
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id)
        REFERENCES league (league_id)
)

'''
#run_command(c1)
run_command(c)
##league_id TEXT, was removed due to unique constraint error

In [60]:
c = '''

INSERT OR IGNORE INTO team
SELECT team_id, league, city, nickname, franch_id
FROM team_codes
'''

run_command(c)

In [61]:
q = '''
SELECT * FROM team
LIMIT 5

'''
run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


`2. Create the game table with columns, primary key, and foreign key as shown in the schema diagram.
 - Select the appropriate type based on the data.
 - Insert the data from the game_log table.
 - Write a query to display the first few rows of the table.

In [65]:
c1 = '''

DROP TABLE IF EXISTS game
'''
c= '''
CREATE TABLE IF NOT EXISTS game(
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs REAL,
    day INTEGER,
    completion TEXT,
    forfeit TEXT,
    protest TEXT,
    attendance REAL,
    length_minutes REAL,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id)
        REFERENCES park (park_id)
)

'''
#run_command(c1)
run_command(c)

In [66]:
c = '''

INSERT OR IGNORE INTO game
SELECT game_id, date, number_of_game, park_id, length_outs, day_night, completion, forefeit,
       protest, attendance, length_minutes, additional_info, acquisition_info
FROM game_log
'''

run_command(c)


In [67]:
q = '''
SELECT * FROM game
LIMIT 5

'''

run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54.0,D,None,None,None,200.0,120.0,None,Y
1,WS3187105050,18710505,0,WAS01,54.0,D,None,None,None,5000.0,145.0,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54.0,D,None,None,None,1000.0,140.0,None,Y
3,CH1187105080,18710508,0,CHI01,54.0,D,None,None,None,5000.0,150.0,None,Y
4,TRO187105090,18710509,0,TRO01,54.0,D,None,None,None,3250.0,145.0,HTBF,Y


1. Create the team_appearance table with columns, primary key, and foreign keys as shown in the schema diagram.
 - Select the appropriate type based on the data.
 - Insert the data from the game_log table, using a UNION clause to combine the data from the column sets for the home and away teams.
 - Write a query to verify that your data was inserted correctly.

In [68]:
q = '''

SELECT sql FROM sqlite_master
WHERE name = "game_log"
    AND type = "table";
'''
run_query(q)

sql
0  CREATE TABLE "game_log" (\n"date" INTEGER,\n  "number_of_game" INTEGER,\n  "day_of_week" TEXT,\n  "v_name" TEXT,\n  "v_league" TEXT,\n  "v_game_number" INTEGER,\n  "h_name" TEXT,\n  "h_league" TEXT,\n  "h_game_number" INTEGER,\n  "v_score" INTEGER,\n  "h_score" INTEGER,\n  "length_outs" REAL,\n  "day_night" TEXT,\n  "completion" TEXT,\n  "forefeit" TEXT,\n  "protest" TEXT,\n  "park_id" TEXT,\n  "attendance" REAL,\n  "length_minutes" REAL,\n  "v_line_score" TEXT,\n  "h_line_score" TEXT,\n  "v_at_bats" REAL,\n  "v_hits" REAL,\n  "v_doubles" REAL,\n  "v_triples" REAL,\n  "v_homeruns" REAL,\n  "v_rbi" REAL,\n  "v_sacrifice_hits" REAL,\n  "v_sacrifice_flies" REAL,\n  "v_hit_by_pitch" REAL,\n  "v_walks" REAL,\n  "v_intentional_walks" REAL,\n  "v_strikeouts" REAL,\n  "v_stolen_bases" REAL,\n  "v_caught_stealing" REAL,\n  "v_grounded_into_double" REAL,\n  "v_first_catcher_interference" REAL,\n  "v_left_on_base" REAL,\n  "v_pitchers_used" REAL,\n  "v_individual_earned_runs" REAL,\n  "v_team_earned_runs" REAL,\n  "v_wild_pitches" REAL,\n  "v_balks" REAL,\n  "v_putouts" REAL,\n  "v_assists" REAL,\n  "v_errors" REAL,\n  "v_passed_balls" REAL,\n  "v_double_plays" REAL,\n  "v_triple_plays" REAL,\n  "h_at_bats" REAL,\n  "h_hits" REAL,\n  "h_doubles" REAL,\n  "h_triples" REAL,\n  "h_homeruns" REAL,\n  "h_rbi" REAL,\n  "h_sacrifice_hits" REAL,\n  "h_sacrifice_flies" REAL,\n  "h_hit_by_pitch" REAL,\n  "h_walks" REAL,\n  "h_intentional_walks" REAL,\n  "h_strikeouts" REAL,\n  "h_stolen_bases" REAL,\n  "h_caught_stealing" REAL,\n  "h_grounded_into_double" REAL,\n  "h_first_catcher_interference" REAL,\n  "h_left_on_base" REAL,\n  "h_pitchers_used" REAL,\n  "h_individual_earned_runs" REAL,\n  "h_team_earned_runs" REAL,\n  "h_wild_pitches" REAL,\n  "h_balks" REAL,\n  "h_putouts" REAL,\n  "h_assists" REAL,\n  "h_errors" REAL,\n  "h_passed_balls" REAL,\n  "h_double_plays" REAL,\n  "h_triple_plays" REAL,\n  "hp_umpire_id" TEXT,\n  "hp_umpire_name" TEXT,\n  "1b_umpire_id" TEXT,\n  "1b_umpire_name" TEXT,\n  "2b_umpire_id" TEXT,\n  "2b_umpire_name" TEXT,\n  "3b_umpire_id" TEXT,\n  "3b_umpire_name" TEXT,\n  "lf_umpire_id" TEXT,\n  "lf_umpire_name" TEXT,\n  "rf_umpire_id" TEXT,\n  "rf_umpire_name" TEXT,\n  "v_manager_id" TEXT,\n  "v_manager_name" TEXT,\n  "h_manager_id" TEXT,\n  "h_manager_name" TEXT,\n  "winning_pitcher_id" TEXT,\n  "winning_pitcher_name" TEXT,\n  "losing_pitcher_id" TEXT,\n  "losing_pitcher_name" TEXT,\n  "saving_pitcher_id" TEXT,\n  "saving_pitcher_name" TEXT,\n  "winning_rbi_batter_id" TEXT,\n  "winning_rbi_batter_id_name" TEXT,\n  "v_starting_pitcher_id" TEXT,\n  "v_starting_pitcher_name" TEXT,\n  "h_starting_pitcher_id" TEXT,\n  "h_starting_pitcher_name" TEXT,\n  "v_player_1_id" TEXT,\n  "v_player_1_name" TEXT,\n  "v_player_1_def_pos" REAL,\n  "v_player_2_id" TEXT,\n  "v_player_2_name" TEXT,\n  "v_player_2_def_pos" REAL,\n  "v_player_3_id" TEXT,\n  "v_player_3_name" TEXT,\n  "v_player_3_def_pos" REAL,\n  "v_player_4_id" TEXT,\n  "v_player_4_name" TEXT,\n  "v_player_4_def_pos" REAL,\n  "v_player_5_id" TEXT,\n  "v_player_5_name" TEXT,\n  "v_player_5_def_pos" REAL,\n  "v_player_6_id" TEXT,\n  "v_player_6_name" TEXT,\n  "v_player_6_def_pos" REAL,\n  "v_player_7_id" TEXT,\n  "v_player_7_name" TEXT,\n  "v_player_7_def_pos" REAL,\n  "v_player_8_id" TEXT,\n  "v_player_8_name" TEXT,\n  "v_player_8_def_pos" REAL,\n  "v_player_9_id" TEXT,\n  "v_player_9_name" TEXT,\n  "v_player_9_def_pos" REAL,\n  "h_player_1_id" TEXT,\n  "h_player_1_name" TEXT,\n  "h_player_1_def_pos" REAL,\n  "h_player_2_id" TEXT,\n  "h_player_2_name" TEXT,\n  "h_player_2_def_pos" REAL,\n  "h_player_3_id" TEXT,\n  "h_player_3_name" TEXT,\n  "h_player_3_def_pos" REAL,\n  "h_player_4_id" TEXT,\n  "h_player_4_name" TEXT,\n  "h_player_4_def_pos" REAL,\n  "h_player_5_id" TEXT,\n  "h_player_5_name" TEXT,\n  "h_player_5_def_pos" REAL,\n  "h_player_6_id" TEXT,\n  "h_player_6_name" TEXT,\n  "h_player_6_def_pos" REAL,\n  "h_player_7_id" TEXT,\n  "h_player_7_name" TEXT,\n  "h_playe

In [72]:
c1 = '''

DROP TABLE IF EXISTS team_appearance
'''

c= '''
CREATE TABLE IF NOT EXISTS team_appearance(
    team_id TEXT,
    game_id TEXT,
    home INTEGER,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats REAL,
    hits REAL,
    doubles REAL,
    triples REAL,
    homeruns REAL,
    rbi REAL,
    sacrifice_hits REAL,
    sacrifice_flies REAL,
    hit_by_pitch REAL,
    walks REAL,
    intentional_walks REAL,
    strikeouts REAL,
    stolen_bases REAL,
    caught_stealing REAL,
    grounded_into_double REAL,
    first_catcher_interference REAL,
    left_on_base REAL,
    pitchers_used REAL,
    individual_earned_runs REAL,
    team_earned_runs REAL,
    wild_pitches REAL,
    balks REAL,
    putouts REAL,
    assists REAL,
    errors REAL,
    passed_balls REAL,
    double_plays REAL,
    triple_plays REAL,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id)
        REFERENCES team (team_id),
    FOREIGN KEY (game_id)
        REFERENCES game (game_id),
    FOREIGN KEY (league_id)
        REFERENCES league (league_id)
)

'''
#run_command(c1)
run_command(c)

In [73]:
c = '''

INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi REAL,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi REAL,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from game_log;

'''

run_command(c)

In [74]:
q = '''
SELECT * FROM team_appearance
WHERE hits NOT NULL
LIMIT 5

'''

run_query(q)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ANA,ANA200004030,1,AL,2,010000001,35.0,10.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,5.0,0.0,6.0,0.0,1.0,0.0,0.0,11.0,3.0,3.0,3.0,0.0,0.0,27.0,16.0,1.0,0.0,0.0,0.0
1,ANA,ANA200004040,1,AL,3,000003000,36.0,10.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,6.0,0.0,9.0,1.0,0.0,2.0,0.0,12.0,4.0,5.0,5.0,0.0,0.0,27.0,12.0,1.0,0.0,2.0,0.0
2,ANA,ANA200004050,1,AL,12,12610110x,33.0,12.0,4.0,0.0,1.0,12.0,0.0,2.0,1.0,8.0,0.0,4.0,1.0,0.0,0.0,0.0,8.0,3.0,6.0,6.0,0.0,0.0,27.0,13.0,2.0,0.0,2.0,0.0
3,ANA,ANA200004070,1,AL,7,30000310x,32.0,9.0,2.0,0.0,2.0,7.0,1.0,0.0,0.0,5.0,2.0,5.0,3.0,0.0,0.0,0.0,7.0,2.0,3.0,3.0,0.0,0.0,27.0,9.0,0.0,0.0,1.0,0.0
4,ANA,ANA200004080,1,AL,7,20000401x,35.0,13.0,4.0,0.0,1.0,7.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,0.0,6.0,6.0,5.0,5.0,2.0,0.0,27.0,18.0,0.0,0.0,3.0,0.0


1. Create the person_appearance table with columns, primary key, and foreign keys as shown in the schema diagram.
 - Select the appropriate type based on the data.
 - Insert the data from the game_log table, using UNION clauses to combine the data from the columns for managers, umpires, pitchers, and awards.
 - Use a loop with string formatting to insert the data for offensive and defensive positions from the game_log table.
 - Write a query to verify that your data was inserted correctly.

In [82]:
c0 = "DROP TABLE IF EXISTS person_appearance"

#run_command(c0)

c1 = """
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

c2 = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
"""

template = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

run_command(c1)
run_command(c2)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [84]:
q = '''
SELECT * FROM person_appearance
LIMIT 5

'''
run_query(q)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1,maplb901,None,ALT188404300,UHP
1,2,curte801,ALT,ALT188404300,MM
2,3,murpj104,ALT,ALT188404300,PSP
3,4,hodnc101,SLU,ALT188404300,PSP
4,5,sullt101,SLU,ALT188404300,MM


Drop the tables we created to hold our unnormalized data:

    - game_log.
    - park_codes.
    - team_codes.
    - person_codes.


In [85]:
c1 = '''
DROP TABLE IF EXISTS game_log

'''

c2 = '''
DROP TABLE IF EXISTS park_codes

'''

c3 = '''
DROP TABLE IF EXISTS team_codes

'''

c4 = '''
DROP TABLE IF EXISTS person_codes

'''

run_command(c1)
run_command(c2)
run_command(c3)
run_command(c4)

In [86]:
show_tables()

,name,type
0,person,table
1,park,table
2,appearance_type,table
3,league,table
4,team,table
5,game,table
6,team_appearance,table
7,person_appearance,table
